In [ ]:
import numpy as np
import pandas
import math
import matplotlib.pyplot as plt
from matplotlib import ticker
import os

In [ ]:
# set the input parameters
eta_QD = 1
eta_EMF = 2
eta_SSET = 1

mu_Singlet_Sensitiser = 1000 # attenuation coefficient base e (cm-1)

solution_conc_TIPS_Tc = 200 # mg/mL
solution_conc_QD = 100 # mg/mL
mass_quotient = solution_conc_QD/solution_conc_TIPS_Tc

solution_to_solid_ratio = 140 # convert solution mass attenutation coeff to solid state attenutation coeff

def calc_f(solution_conc_QD, solution_conc_TIPS_Tc):
    """ calculate the QD mass fraction from the precurser solution concentrations
    args: 
    solution_conc_QD = the solution conc for the QD
    solution_conc_TIPS_Tc = the solution conc for the TIPS-Tc
    outputs:
    the QD mass fraction
    """
    ans = solution_conc_QD/(solution_conc_QD + solution_conc_TIPS_Tc)
    return ans

f = calc_f(solution_conc_QD, solution_conc_TIPS_Tc)

In [ ]:
# load absorption data

def load_absorption_data(data_path, ratio):
    df = pandas.read_csv(data_path)
    df['ratio'] = ratio
    df['Attenuation Coeff base 10 (cm-1)'] = \
        df['Mass Attenuation Coeff (Lg-1cm-1)']*df['ratio']
    df['Attenuation Coeff (cm-1)'] = \
        df['Attenuation Coeff base 10 (cm-1)']*np.log(10)
    return df

def load_absorption_test():
    TIPS_Tc_data_path = 'Experimental_Data\Mass Attenutation Coeffs TIPS-Tc.csv'
    TIPS_Tc_data = pandas.read_csv(TIPS_Tc_data_path)
    TIPS_Tc_data['ratio']=solution_to_solid_ratio
    TIPS_Tc_data['Attenuation Coeff base 10 (cm-1)'] = TIPS_Tc_data['Mass Attenuation Coeff (Lg-1cm-1)']*TIPS_Tc_data['ratio']
    TIPS_Tc_data['Attenuation Coeff (cm-1)'] = TIPS_Tc_data['Attenuation Coeff base 10 (cm-1)']*np.log(10)
    test_value = np.array(TIPS_Tc_data[:10])
    value = np.array(load_absorption_data(TIPS_Tc_data_path, solution_to_solid_ratio)[:10])
    assert np.all((test_value - value)==0)

TIPS_Tc_data_path = 'Experimental_Data\Mass Attenutation Coeffs TIPS-Tc.csv'
TIPS_Tc_data = load_absorption_data(TIPS_Tc_data_path, solution_to_solid_ratio)

QD_data_path = 'Experimental_Data\Mass Attenutation Coeffs PbS-TET-CA.csv'
QD_data = load_absorption_data(QD_data_path, solution_to_solid_ratio*mass_quotient)

In [ ]:
# set the varaiables
wl = np.array(QD_data['Wavelength (nm)'])


In [ ]:
# extraction absorption ratios
mu_SF = np.array(TIPS_Tc_data['Attenuation Coeff (cm-1)'])
mu_QD = np.array(QD_data['Attenuation Coeff (cm-1)'])
mu_Sen = mu_Singlet_Sensitiser*((wl<=450).astype(float))
mu_total = mu_SF + mu_QD + mu_Sen
alpha_SF = mu_SF/mu_total
alpha_QD = mu_QD/mu_total
alpha_Sen = mu_Sen/mu_total

In [ ]:
def Calc_eta_PM(eta_QD,eta_SSET, eta_EMF, alpha_QD, alpha_SF, alpha_Sen):
    """ calculates the wavelength dependent eta_PM, the SF_PM PLQE
    """
    return eta_QD*( alpha_QD + (alpha_SF + alpha_Sen*eta_SSET)*eta_EMF )

def Calc_eta_PM_test1():
    # no sf absorption condition
    test_alpha_QD = np.ones(100)
    test_alpha_SF = np.zeros(100)
    test_eta_QD = 1
    test_eta_EMF = 2
    assert Calc_eta_PM(test_eta_QD, test_alpha_QD,test_alpha_SF,test_eta_EMF) == np.ones(100)

def Calc_eta_PM_test1():
    # no QD absorption condition
    test_alpha_QD = np.zeros(100)
    test_alpha_SF = np.ones(100)
    test_eta_QD = 1
    test_eta_EMF = 2
    assert Calc_eta_PM(test_eta_QD, test_alpha_QD,test_alpha_SF,test_eta_EMF) == 2*np.ones(100)



In [ ]:
# calculate the PLQE of the SFPM
eta_PM = Calc_eta_PM(eta_QD,eta_SSET, eta_EMF, alpha_QD, alpha_SF, alpha_Sen)

In [ ]:
# plot the PLQE vs Wavelength

Blue = '#046DE0'
Yellow = '#FDBE3D'
Green = '#38B99E'
Dark_Grey = '#515151'


fig, ax = plt.subplots(1,1)
ax.plot(wl,eta_PM, color = Green,ls = "-",lw=5)

ax.set_xlim(400,650)
ax.set_ylim(0,2)
ax.set_xlabel('Wavelength (nm)')
ax.set_ylabel(r'$\eta_{PM}$')
yticker = ticker.MultipleLocator(0.5)
ax.yaxis.set_major_locator(yticker)
xticker = ticker.MultipleLocator(100)
ax.xaxis.set_major_locator(xticker)

In [ ]:
# extraction absorption ratios
mu_SF = np.array(TIPS_Tc_data['Attenuation Coeff (cm-1)'])
mu_QD = np.array(QD_data['Attenuation Coeff (cm-1)'])
n = len(mu_SF)

# amplitude varaiable for the sensitiser
min_log10_amp = -5
max_log10_amp = 5
step_log10_amp = 0.1
n_steps_amp = (max_log10_amp-min_log10_amp)/step_log10_amp
amp = 10**np.linspace(min_log10_amp,max_log10_amp,n_steps_amp+1)
m = len(amp)

# set up the mu matrices
mu_SF = np.reshape(mu_SF,(n,1)) @ np.ones((1,m))
mu_QD = np.reshape(mu_QD,(n,1)) @ np.ones((1,m))

mu_Sen = mu_Singlet_Sensitiser*((wl<=450).astype(float))
mu_Sen = np.reshape(mu_Sen,(n,1)) @ np.reshape(amp,(1,m))

# calculate the total mu and the fraction absorptions
mu_total = mu_SF + mu_QD + mu_Sen
alpha_SF = mu_SF/mu_total
alpha_QD = mu_QD/mu_total
alpha_Sen = mu_Sen/mu_total


In [ ]:
# calculate the PLQE of the SFPM
eta_PM = Calc_eta_PM(eta_QD,eta_SSET, eta_EMF, alpha_QD, alpha_SF, alpha_Sen)

In [ ]:
# plot the PLQE vs Wavelength

fig, ax = plt.subplots(1,1)
pcm = ax.pcolormesh(wl,alpha_Sen[-1,:],eta_PM.T,\
    vmin = 0, vmax = 2\
    )
fig.colorbar(pcm,ax=ax)
#ax.set_yscale('log')

ax.set_xlim(400,650)
ax.set_ylim(0,1)

ax.set_xlabel('Wavelength (nm)')
ax.set_ylabel(r'$\alpha_{Sen}$')
#yticker = ticker.MultipleLocator(0.5)
#ax.yaxis.set_major_locator(yticker)
xticker = ticker.MultipleLocator(100)
ax.xaxis.set_major_locator(xticker)

In [ ]:
# investigate a particular wavelength
ind =  1167 # 422 nm
ind = 1184 # 405 nm
print('displaying the PM efficiency at ',wl[ind],' nm.')

x=alpha_Sen[ind,:]
y=eta_PM[ind,:]

fig, ax = plt.subplots(1,1)
ax.plot(x,y, color = Green,ls = "-",lw=5)

ax.set_xlim(0,1)
ax.set_ylim(0,2)
ax.set_xlabel(r'$\alpha_{Sen}$')
ax.set_ylabel(r'$\eta_{PM}$')
yticker = ticker.MultipleLocator(0.5)
ax.yaxis.set_major_locator(yticker)
xticker = ticker.MultipleLocator(0.2)
ax.xaxis.set_major_locator(xticker)

In [ ]:
# investigate a particular wavelength
ind =  1167 # 422 nm
ind = 1184 # 405 nm
print('displaying the PM efficiency at ',wl[ind],' nm.')

x=mu_Sen[ind,:]
y=eta_PM[ind,:]

fig, ax = plt.subplots(1,1)
ax.plot(x,y, color = Green,ls = "-",lw=5)

ax.set_xscale('log')
ax.set_xlim(10,100000)
ax.set_ylim(0,2)
ax.set_xlabel(r'$\mu_{Sen}$')
ax.set_ylabel(r'$\eta_{PM}$')
yticker = ticker.MultipleLocator(0.5)
ax.yaxis.set_major_locator(yticker)
#xticker = ticker.MultipleLocator(0.2)
#ax.xaxis.set_major_locator(xticker)